<a href="https://colab.research.google.com/github/ftnext/practice-dl-nlp/blob/master/bert_exercise/modernBERT_synthetic_domain_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/argilla-io/synthetic-data-generator/blob/5a40c394b8aa9dc7ed21451f6c7db2bcdff7f13d/examples/fine-tune-modernbert-classifier.ipynb

In [20]:
!nvidia-smi

Thu Jun  5 13:55:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P0             29W /   70W |    5966MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!pip install -q transformers

In [2]:
!pip install -q -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [3]:
!pip uninstall -y wandb

Found existing installation: wandb 0.19.11
Uninstalling wandb-0.19.11:
  Successfully uninstalled wandb-0.19.11


In [4]:
!pip list

Package                               Version
------------------------------------- -------------------
absl-py                               1.4.0
accelerate                            1.7.0
aiofiles                              24.1.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.11.15
aiosignal                             1.3.2
alabaster                             1.0.0
albucore                              0.0.24
albumentations                        2.0.8
ale-py                                0.11.1
altair                                5.5.0
annotated-types                       0.7.0
antlr4-python3-runtime                4.9.3
anyio                                 4.9.0
argon2-cffi                           23.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.2
arviz                                 0.21.0
astropy                               7.1.0
astropy-iers-data                     0.2025.5.26.0.

## Fine-tune

In [5]:
import numpy as np
from datasets import load_dataset
from sklearn.metrics import f1_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer

In [6]:
train_dataset = load_dataset("argilla/synthetic-domain-text-classification", split="train")
split_dataset = train_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)
split_dataset["train"][0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/3.24k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/336k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'text': "In recent years, the manufacturing sector has seen a significant shift towards automation and digitalization. Companies across the globe are investing heavily in advanced technologies such as robotics and artificial intelligence to streamline their production processes and improve efficiency. This trend is not limited to major corporations but also includes small and medium-sized businesses that aim to stay competitive in the market. In addition, there's been a surge of startups focusing on innovative solutions designed specifically for industrial applications.",
 'label': 0}

In [7]:
print(len(split_dataset["train"]), len(split_dataset["test"]))

900 100


In [8]:
model_id = "answerdotai/ModernBERT-base"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, return_tensors="pt")

if "label" in split_dataset["train"].features.keys():
    split_dataset = split_dataset.rename_column("label", "labels")
tokenized_dataset = split_dataset.map(tokenize, batched=True, remove_columns=["text"])

tokenized_dataset["train"].features.keys()

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

dict_keys(['labels', 'input_ids', 'attention_mask'])

In [10]:
labels = tokenized_dataset["train"].features["labels"].names
num_labels = len(labels)
label2id, id2label = {}, {}
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [11]:
print(f"{label2id=}")
print(f"{id2label=}")

label2id={'business-and-industrial': '0', 'books-and-literature': '1', 'beauty-and-fitness': '2', 'autos-and-vehicles': '3', 'people-and-society': '4', 'sports': '5', 'shopping': '6', 'online-communities': '7', 'pets-and-animals': '8', 'internet-and-telecom': '9', 'home-and-garden': '10', 'adult': '11', 'science': '12', 'food-and-drink': '13', 'real-estate': '14', 'news': '15', 'jobs-and-education': '16', 'health': '17', 'hobbies-and-leisure': '18', 'games': '19', 'computers-and-electronics': '20', 'arts-and-entertainment': '21', 'travel-and-transportation': '22', 'finance': '23', 'law-and-government': '24', 'sensitive-subjects': '25'}
id2label={'0': 'business-and-industrial', '1': 'books-and-literature', '2': 'beauty-and-fitness', '3': 'autos-and-vehicles', '4': 'people-and-society', '5': 'sports', '6': 'shopping', '7': 'online-communities', '8': 'pets-and-animals', '9': 'internet-and-telecom', '10': 'home-and-garden', '11': 'adult', '12': 'science', '13': 'food-and-drink', '14': 'rea

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label
)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    score = f1_score(labels, predictions, labels=labels, average="weighted")
    return {"f1": float(score) if score == 1 else score}

In [14]:
training_args = TrainingArguments(
    output_dir="ModernBERT-domain-classifier",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=3,
    bf16=True,
    optim="adamw_torch_fused",
    logging_strategy="steps",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

W0605 13:47:32.537000 444 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.788800,0.400068,0.902423
2,0.356500,0.286747,0.909235
3,0.068000,0.150512,0.931303


TrainOutput(global_step=339, training_loss=0.6052564463432559, metrics={'train_runtime': 378.816, 'train_samples_per_second': 7.127, 'train_steps_per_second': 0.895, 'total_flos': 672176807359200.0, 'train_loss': 0.6052564463432559, 'epoch': 3.0})

In [17]:
trainer.evaluate()

{'eval_loss': 0.15051162242889404,
 'eval_f1': 0.9313029990427355,
 'eval_runtime': 2.0528,
 'eval_samples_per_second': 48.714,
 'eval_steps_per_second': 6.333,
 'epoch': 3.0}

## Inference

In [18]:
from transformers import pipeline

classifier = pipeline(
    task="text-classification",
    model="/content/ModernBERT-domain-classifier/checkpoint-339",
    tokenizer=tokenizer,
    device=0
)

Device set to use cuda:0


In [19]:
classifier("Smoking is bad for your health.")

[{'label': 'health', 'score': 0.8847112059593201}]